In [1]:
import torch

In [2]:
import transformers

In [3]:
from transformers import BertTokenizer,BertForNextSentencePrediction

In [4]:
tokenizer=BertTokenizer.from_pretrained("bert-base-uncased")
model=BertForNextSentencePrediction.from_pretrained("bert-base-uncased")

In [5]:
with open("clean.txt","r") as fp:
    text=fp.read().split("\n")

In [6]:
text[:3]

['From my grandfather Verus I learned good morals and the government of my temper.',
 'From the reputation and remembrance of my father, modesty and a manly character.',
 'From my mother, piety and beneficence, and abstinence, not only from evil deeds, but even from evil thoughts; and further, simplicity in my way of living, far removed from the habits of the rich.']

In [7]:
for i in text[51:55]:
    print(i.split("."))
    print(len(i.split(".")))
    break

['Body, soul, intelligence: to the body belong sensations, to the soul appetites, to the intelligence principles', ' To receive the impressions of forms by means of appearances belongs even to animals; to be pulled by the strings of desire belongs both to wild beasts and to men who have made themselves into women, and to a Phalaris and a Nero: and to have the intelligence that guides to the things which appear suitable belongs also to those who do not believe in the gods, and who betray their country, and do their impure deeds when they have shut the doors', ' If then everything else is common to all that I have mentioned, there remains that which is peculiar to the good man, to be pleased and content with what happens, and with the thread which is spun for him; and not to defile the divinity which is planted in his breast, nor disturb it by a crowd of images, but to preserve it tranquil, following it obediently as a god, neither saying anything contrary to the truth, nor doing anythin

In [8]:
bag =[sentences for paragraph in text for sentences in paragraph.split(".") if sentences!=""]

In [9]:
len(bag)

1372

In [10]:
import random

sentence_a=[]
sentence_b=[]
label=[]

for paragraph in text:
    sentences=[
        sentence for sentence in paragraph.split(".") if sentence!=""
    ]
    num_sentences=len(sentences)
    
    if num_sentences > 1:
        start=random.randint(0,num_sentences-2)
        sentence_a.append(sentences[start])
        if random.random()>0.5:
            sentence_b.append(bag[random.randint(0,len(bag)-1)])
            label.append(1)
        else:
            sentence_b.append(sentences[start+1])
            label.append(0)

In [11]:
len(sentence_a)

317

In [12]:
len(sentence_b)

317

In [13]:
len(label)

317

In [14]:
sentence__a=sentence_a[:10]
sentence__b=sentence_b[:10]
label_=label[:10]

In [15]:
len(label_)

10

In [16]:
inputs=tokenizer(sentence__a,sentence__b,return_tensors='pt',padding=True,truncation=True)

In [17]:
inputs.input_ids.shape

torch.Size([10, 253])

In [18]:
inputs["labels"]=torch.LongTensor([label_]).T

In [19]:
inputs.labels.shape

torch.Size([10, 1])

In [20]:
inputs

{'input_ids': tensor([[  101,  2002,  2001,  ...,     0,     0,     0],
        [  101,  1045,  5159,  ...,     0,     0,     0],
        [  101,  2582,  1010,  ...,  1037,  7786,   102],
        ...,
        [  101,  1998, 15223,  ...,     0,     0,     0],
        [  101,  2079,  3308,  ...,     0,     0,     0],
        [  101,  2021,  2059,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 1, 1, 1],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [1]])}

# Custom training

In [47]:
class CustomDataSet(torch.utils.data.Dataset):
    def __init__(self,data):
        self.data=data
        
    def __len__(self):
        return self.data.input_ids.shape[0]
    
    def __getitem__(self,idx):
        sample={
            key: torch.Tensor(value[idx]) for key,value in self.data.items()
        }
        return sample

In [48]:
dataset=CustomDataSet(inputs)

In [49]:
loader=torch.utils.data.DataLoader(dataset,batch_size=2,shuffle=True)

In [50]:
from tqdm import tqdm
from torch.optim import AdamW

In [52]:
model.train()

BertForNextSentencePrediction(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [53]:
optim=AdamW(model.parameters(),lr=5e-5)

In [54]:
epochs=2

In [57]:
for epoch in range(epochs):
    loop=tqdm(loader,leave=True)
    for batch in loop:
        optim.zero_grad()
        token_ids=batch["input_ids"]
        token_type_ids=batch["token_type_ids"]
        attention_mask=batch["attention_mask"]
        labels=batch["labels"]
        op=model(token_ids,token_type_ids=token_type_ids,attention_mask=attention_mask,labels=labels)
        loss=op.loss
        loss.backward()
        optim.step()
        
        loop.set_description(f"Epoch :{epoch}")
        loop.set_postfix(loss=loss.item())

Epoch :1: 100%|███████████████████████████████████████████████████████████████| 5/5 [01:34<00:00, 18.99s/it, loss=1.17]


In [58]:
with torch.no_grad():
    opp=model(**inputs)

In [59]:
opp.keys()

odict_keys(['loss', 'logits'])

In [60]:
opp.loss

tensor(0.0098)

In [64]:
len(opp.logits)

10

In [70]:
for i in range(len(opp.logits)):
    print("original:",inputs.labels[i])
    print("predicted:",torch.argmax(opp.logits[i]))

original: tensor([0])
predicted: tensor(0)
original: tensor([0])
predicted: tensor(0)
original: tensor([1])
predicted: tensor(1)
original: tensor([0])
predicted: tensor(0)
original: tensor([1])
predicted: tensor(1)
original: tensor([0])
predicted: tensor(0)
original: tensor([1])
predicted: tensor(1)
original: tensor([0])
predicted: tensor(0)
original: tensor([1])
predicted: tensor(1)
original: tensor([1])
predicted: tensor(1)


# Direct method(inbuilt)

In [21]:
op=model(**inputs)

In [22]:
op.logits.shape

torch.Size([10, 2])

In [25]:
op.loss

tensor(4.5053, grad_fn=<NllLossBackward0>)

In [26]:
for i in range(len(op.logits)):
    print("original:\n",inputs.labels[i].item())
    print("predicted:",torch.argmax(op.logits[i]).item())

original:
 1
predicted: 0
original:
 1
predicted: 0
original:
 0
predicted: 0
original:
 0
predicted: 0
original:
 0
predicted: 0
original:
 0
predicted: 0
original:
 1
predicted: 0
original:
 0
predicted: 0
original:
 1
predicted: 0
original:
 1
predicted: 0
